In [1]:
from os import environ
if environ.get('netlogo_path') is not None:
    netlogo_path = environ.get('netlogo_path')
else:
    netlogo_path = "C:/Program Files/NetLogo 6.0.2/"
import nl4py
# Replace argument to startServer(...) with the location of your NetLogo installation 
nl4py.startServer(netlogo_path)
import time
def simulate(workspace_):
    workspace_.command("stop")
    # Random initialization of parameters 
    # Same netlogo commands as used for the PyNetLogo evaluation
    workspace_.command("set initial-number-sheep random-float 250")
    workspace_.command("set initial-number-wolves random-float 250")
    workspace_.command("set grass-regrowth-time random-float 100")
    workspace_.command("set sheep-gain-from-food random-float 50")
    workspace_.command("set wolf-gain-from-food random 100")
    workspace_.command("set sheep-reproduce random-float 20")
    workspace_.command("set wolf-reproduce random-float 20")
    workspace_.command("set show-energy? false")
    workspace_.command('set model-version "sheep-wolves-grass"')
    workspace_.command('setup')
    workspace_.scheduleReportersAndRun(["ticks",'count sheep','count wolves'], 0,1,100,"go")    

Dependencies installed successfully! 
Start the NetLogoControllerServer using nl4py.startServer()
Shutting down old server instance...
Server started.


In [2]:
def measureExecutionTime(runsNeeded):
    startTime = int(round(time.time() * 1000))    
    runsDone = 0
    runsStarted = 0
    allResults = []
    # Make sure we start 8 headless workspaces to compare to 
    # 8 IPCluster engines running PyNetLogo
    for i in range(0,8):
        workspace = nl4py.newNetLogoHeadlessWorkspace()
        workspace.openModel('../Models/Wolf Sheep Predation.nlogo')
        simulate(workspace)
        runsStarted = runsStarted + 1
    while (runsDone < runsNeeded):
        for workspace in nl4py.getAllHeadlessWorkspaces():
            newResults = workspace.getScheduledReporterResults()
            # Make sure the models have finished running.
            if len(newResults) > 0:
                allResults.extend(newResults)
                runsDone = runsDone + 1
                if runsStarted < runsNeeded:
                    simulate(workspace)
                    runsStarted = runsStarted + 1
    stopTime = int(round(time.time() * 1000))
    executionTime = stopTime - startTime
    return executionTime

In [4]:
import os
outputFile = '../output/5.3_output.csv'
out = open (outputFile, "a+")

In [3]:
for j in range (0,10):
    runs = 5000
    executionTime = measureExecutionTime(runs)
    print("nl4py,scheduledReporters," + str(runs) + "," + str(executionTime))
    out.write("nl4py,scheduledReporters," + str(runs) + "," + str(executionTime) + "\n")
    nl4py.deleteAllHeadlessWorkspaces()
    out.flush()
    runs = 10000
    executionTime = measureExecutionTime(runs)
    print("nl4py,scheduledReporters," + str(runs) + "," + str(executionTime))
    out.write("nl4py,scheduledReporters," + str(runs) + "," + str(executionTime) + "\n")
    nl4py.deleteAllHeadlessWorkspaces() 
    out.flush()
    runs = 15000
    executionTime = measureExecutionTime(runs)
    print("nl4py,scheduledReporters," + str(runs) + "," + str(executionTime))
    out.write("nl4py,scheduledReporters," + str(runs) + "," + str(executionTime) + "\n")
    nl4py.deleteAllHeadlessWorkspaces()
    out.flush()

214858
441019
651024
226374
459934
673229
223660
460537
696373
234685
475331
718277
241410
470090
707271
245718
493253
714512
231326
484965
720576
239841
487072
733894
244353
494435
742279
250483
490103
755348


In [5]:
out.close()